In [1]:
import os
import time
import json
from copy import copy

import pandas as pd
import gokart
import luigi
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from mahjong.shanten import Shanten
from mahjong.hand_calculating.hand import HandCalculator
from mahjong.tile import TilesConverter
from mahjong.hand_calculating.hand_config import HandConfig
from mahjong.meld import Meld

from mleagueloader import MLEAGUELoader, format_raw_data, tehai_shanten, TehaiGenerator

In [2]:
DEBUG = True

In [3]:
webdriver.__version__

'4.6.0'

In [4]:
pd.options.display.max_rows = 200
pd.options.display.max_colwidth=80

In [5]:
load_dotenv()

MLEAGUE_EMAIL = os.environ['MLEAGUE_EMAIL']
MLEAGUE_PASS=os.environ['MLEAGUE_PASS']

In [6]:
class LoaderTask(gokart.TaskOnKart):
    season = luigi.IntParameter(default=2022)
    debug = luigi.BoolParameter(default=True)

    def run(self):
        options = Options()

        # Headless
        options.headless = False

        # ブラウザ起動
        driver = webdriver.Chrome(executable_path="chromedriver.exe", options=options)
        loader = MLEAGUELoader(driver=driver)

        loader.login_fun(MLEAGUE_EMAIL, MLEAGUE_PASS)

        # 2022シーズンの終了した試合を取得
        schedule = loader.get_match_schedule(self.season)
        schedule.sort(reverse=False)

        if self.debug:
            schedule = schedule[:2]

        df = loader.load(schedule)
        self.dump(df)

In [7]:
df = gokart.build(LoaderTask())

In [8]:
game_df, user_df = format_raw_data(df)

/Users/hiromu.nakamura/ghq/github.com/po3rin/mleagueloader/mleagueloader/format.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_df['player'] = player_df.apply(lambda x: x['args'][0] if x['cmd'] in player_action else None, axis=1)
/Users/hiromu.nakamura/ghq/github.com/po3rin/mleagueloader/mleagueloader/format.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_df['name'] = player_df.apply(lambda x: x['args'][1] if x['cmd'] in player_action else None, axis=1)
/Users/hiromu.nakamura/ghq/git

In [9]:
# game_df.head(200)

In [10]:
# game_df.dtypes

In [11]:
# user_df.head(100)

In [16]:
tehaigen = TehaiGenerator()
game_df = tehaigen.generate(game_df)
# game_df.head(200)

game_df[game_df['kyokuid']=='0_D0_0_0_2z_2z_3z_4z_1z'].head(200)

,time,id,cmd,args,gameid,kyokuid,player,hai,tsumogiri,nokori_yama,tehai
2833,2022-10-03 21:20:37,1300000,kyokustart,"[0, D0, 0, 0, 2z, 2z, 3z, 4z, 1z]",L001_S013_0001_02A,0_D0_0_0_2z_2z_3z_4z_1z,None,None,False,None,None
2834,2022-10-03 21:20:37,1300001,point,"[A0, =35900]",L001_S013_0001_02A,0_D0_0_0_2z_2z_3z_4z_1z,A0,None,False,None,None
2835,2022-10-03 21:20:37,1300002,point,"[B0, =3100]",L001_S013_0001_02A,0_D0_0_0_2z_2z_3z_4z_1z,B0,None,False,None,None
2836,2022-10-03 21:20:37,1300003,point,"[C0, =1200]",L001_S013_0001_02A,0_D0_0_0_2z_2z_3z_4z_1z,C0,None,False,None,None
2837,2022-10-03 21:20:37,1300004,point,"[D0, =59800]",L001_S013_0001_02A,0_D0_0_0_2z_2z_3z_4z_1z,D0,None,False,None,None
2839,2022-10-03 21:20:37,1300006,dora,"[1p, 9p]",L001_S013_0001_02A,0_D0_0_0_2z_2z_3z_4z_1z,None,None,False,None,None
2840,2022-10-03 21:20:37,1300007,haipai,"[D0, 2p3p5P6p]",L001_S013_0001_02A,0_D0_0_0_2z_2z_3z_4z_1z,D0,2p3p5P6p,False,None,"[2p, 3p, 5P, 6p]"
2841,2022-10-03 21:20:37,1300008,haipai,"[A0, 2m3m5m8m]",L001_S013_0001_02A,0_D0_0_0_2z_2z_3z_4z_1z,A0,2m3m5m8m,False,None,"[2m, 3m, 5m, 8m]"
2842,2022-10-03 21:20:37,1300009,haipai,"[B0, 2m4m6m9m]",L001_S013_0001_02A,0_D0_0_0_2z_2z_3z_4z_1z,B0,2m4m6m9m,False,None,"[2m, 4m, 6m, 9m]"
2843,2022-10-03 21:20:37,1300010,haipai,"[C0, 1m2m4m5m]",L001_S013_0001_02A,0_D0_0_0_2z_2z_3z_4z_1z,C0,1m2m4m5m,False,None,"[1m, 2m, 4m, 5m]"


In [13]:
# game_df['tehai_shanten'] = game_df['tehai'].apply(lambda x: tehai_shanten(x))

# game_df.head(200)